In [1]:
import os
import pickle
import numpy as np
import random
import pandas as pd
import torch
import torch.nn as nn

In [2]:
from IPython.display import clear_output
from factory.AutoVC import AutoVC
from factory.MetaConv import MetaConv
from factory.MetaDV import MetaDV

In [3]:
ROOT = "train_data_en"
device = "cuda:0"

In [4]:
erroment_uttr_idx = 4
max_uttr_idx = 60
num_speaker = 40

In [5]:
#all_speaker = sorted(next(iter(os.walk(f'{ROOT}')))[1],key=int)[:num_speaker]
all_speaker = next(iter(os.walk(f'{ROOT}')))[1][:num_speaker]

In [6]:
# 評估用
C = torch.load('model/static/metadv_en_openslr.pt').to(device)

In [7]:
# 要用原本訓練的來生聲音
metadata = pickle.load(open(f'{ROOT}/train.pkl', "rb"))[:num_speaker]

In [8]:
from melgan.interface import *
vocoder = MelVocoder(model_name='model/static/multi_speaker')

In [11]:
org = AutoVC(44,256,512,22).to(device)
org.load_state_dict(torch.load("model/autovc_pytorch_org_en.pt", map_location=device))

org_dc = AutoVC(44,256,512,22).to(device)
org_dc.load_state_dict(torch.load("model/autovc_pytorch_org_dc_en.pt", map_location=device))

conv = MetaConv(44,256,512,22).to(device) 
conv.load_state_dict(torch.load("model/autovc_pytorch_meta_conv_final_en.pt", map_location=device))

<All keys matched successfully>

In [12]:
def get_trans_mel(model,sample,emb_org,emb_trg):
    return model(sample.to(device),emb_org.to(device),emb_trg.to(device))[1][0,:,:].detach().cpu()

In [13]:
def get_crop_mel(tmp,len_crop=176):
    if tmp.shape[0] < len_crop:
        pad_size = int(len_crop - tmp.shape[0])
        npad = [(0, 0)] * tmp.ndim
        npad[0] = (0, pad_size)
        tmp = np.pad(tmp, pad_width=npad, mode='constant', constant_values=0)
        melsp = torch.from_numpy(tmp)
            
    elif tmp.shape[0] == len_crop:
        melsp = torch.from_numpy(tmp)
    else:
        left = np.random.randint(0, tmp.shape[0] - len_crop)
        melsp = torch.from_numpy(tmp[left : left + len_crop, :])
    return melsp.to(device)

In [56]:
import torch.nn.functional as F
def get_dv(speaker):
    all_uttr = next(iter(os.walk(f'{ROOT}/{speaker}')))[2][:erroment_uttr_idx]
    _dv = torch.zeros((1,256))
    for uttr in all_uttr:
        mel = torch.from_numpy(np.load(f'{ROOT}/{speaker}/{uttr}')).unsqueeze(0).to(device)
        if mel.shape[1] < 176:
            pad_size = int(176 - mel.shape[1])
            mel = F.pad(mel,(0,0,int(pad_size/2),int(pad_size/2)),'constant',0)
            print(mel.shape)
 
        _dv += C(mel)[1].detach().cpu()
    _dv = _dv/(erroment_uttr_idx+1)
    random_uttr = next(iter(os.walk(f'{ROOT}/{speaker}')))[2][random.randint(erroment_uttr_idx,max_uttr_idx)]
    mel = get_crop_mel(np.load(f'{ROOT}/{speaker}/{random_uttr}')).unsqueeze(0)
    emb_org = _dv.to(device) 
    return mel,emb_org,speaker

In [57]:
all_dv = []
for i,speaker in enumerate(all_speaker):
    all_dv.append(get_dv(speaker))

torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])


In [58]:
len(all_dv) 

40

In [59]:
def get_cos_rc(cos_res):
    rc = 0.0
    rc_ = 0.0
    N = len(cos_res)
    for i in range(N):
        total = sum(cos_res[i][i])
        others = (total - cos_res[i][i][i])/(N-1)
        rc_ += others
        rc += cos_res[i][i][i]
    return rc/N,rc_/N

In [60]:
def get_cos_trans(cos_res):
    trans = []
    trans_ = []
    N = len(cos_res)
    for i in range(N):
        trans_cos = np.sum(np.diagonal(cos_res[i])) - cos_res[i][i][i]
        trans.append(trans_cos / (N-1))
        trans_.append(  (np.sum(cos_res[i]) - np.sum(np.diagonal(cos_res[i]))) / ((N-1)*N))
    
    return sum(trans)/len(trans),sum(trans_)/len(trans_)

In [62]:
cos_reslut_rc_org = np.zeros((num_speaker,num_speaker,num_speaker))
cos_reslut_rc_dc = np.zeros((num_speaker,num_speaker,num_speaker))
#cos_reslut_rc_pool = np.zeros((num_speaker,num_speaker,num_speaker))
cos_reslut_rc_conv = np.zeros((num_speaker,num_speaker,num_speaker)) 
temp = None
for i,data in enumerate(metadata):
    sp_s = data[1]
    source_mel = get_crop_mel(np.load(f"{ROOT}//{data[3]}")).unsqueeze(0)
    emb_org = torch.from_numpy(data[2]).unsqueeze(0)
    print(f"Now Processing --- {sp_s}")
    for j,data in enumerate(metadata):
        sp_o = data[1]
        emb_trg = torch.from_numpy(data[2]).unsqueeze(0)
        trans_mel_org = get_trans_mel(org,source_mel,emb_org,emb_trg).to(device)
        trans_mel_dc = get_trans_mel(org_dc,source_mel,emb_org,emb_trg).to(device)
        #trans_mel_pool = get_trans_mel(pool,source_mel,emb_org,emb_trg).to(device)
        trans_mel_conv = get_trans_mel(conv,source_mel,emb_org,emb_trg).to(device)
        org_dv = C(trans_mel_org)[1]
        dc_dv = C(trans_mel_dc)[1]
        #pool_dv = C(trans_mel_pool)[1]
        conv_dv = C(trans_mel_conv)[1]
        
        if i==j:
            print(f"Reconstruct ---- {sp_s} to {sp_o}")
        else:
            print(f"Trans --- {sp_s} to {sp_o}")
        
        for k,data in enumerate(all_dv):
            _,emb,_ = data
            cos = torch.clamp(nn.functional.cosine_similarity(org_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_org[i][j][k] = cos
            
            cos = torch.clamp(nn.functional.cosine_similarity(dc_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_dc[i][j][k] = cos
            #cos = torch.clamp(nn.functional.cosine_similarity(pool_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            #cos_reslut_rc_pool[i][j][k] = cos
            cos = torch.clamp(nn.functional.cosine_similarity(conv_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_conv[i][j][k] = cos

Now Processing --- 0
Reconstruct ---- 0 to 0
Trans --- 0 to 1
Trans --- 0 to 2
Trans --- 0 to 3
Trans --- 0 to 4
Trans --- 0 to 5
Trans --- 0 to 6
Trans --- 0 to 7
Trans --- 0 to 8
Trans --- 0 to 9
Trans --- 0 to 10
Trans --- 0 to 11
Trans --- 0 to 12
Trans --- 0 to 13
Trans --- 0 to 14
Trans --- 0 to 15
Trans --- 0 to 16
Trans --- 0 to 17
Trans --- 0 to 18
Trans --- 0 to 19
Trans --- 0 to 20
Trans --- 0 to 21
Trans --- 0 to 22
Trans --- 0 to 23
Trans --- 0 to 24
Trans --- 0 to 25
Trans --- 0 to 26
Trans --- 0 to 27
Trans --- 0 to 28
Trans --- 0 to 29
Trans --- 0 to 30
Trans --- 0 to 31
Trans --- 0 to 32
Trans --- 0 to 33
Trans --- 0 to 34
Trans --- 0 to 35
Trans --- 0 to 36
Trans --- 0 to 37
Trans --- 0 to 38
Trans --- 0 to 39
Now Processing --- 1
Trans --- 1 to 0
Reconstruct ---- 1 to 1
Trans --- 1 to 2
Trans --- 1 to 3
Trans --- 1 to 4
Trans --- 1 to 5
Trans --- 1 to 6
Trans --- 1 to 7
Trans --- 1 to 8
Trans --- 1 to 9
Trans --- 1 to 10
Trans --- 1 to 11
Trans --- 1 to 12
Trans --- 

## Ground Truth

In [63]:
cos_gt = np.zeros((num_speaker,num_speaker))
for i,speaker in enumerate(all_speaker):
    mel,_,_ = get_dv(speaker)
    _dv = C(mel)[1].detach().cpu()
    for j,data in enumerate(all_dv):
        dv = data[1].detach().cpu()
        cos = torch.clamp(nn.functional.cosine_similarity(_dv,dv, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
        cos_gt[i][j] = cos
np.diagonal(cos_gt).mean(), (np.sum(cos_gt) - (np.sum(np.diagonal(cos_gt)))) /(num_speaker*(num_speaker-1))

torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])


(0.9265956819057465, 0.7373467485874127)

## Reconstruct

In [64]:
get_cos_rc(cos_reslut_rc_org)

(0.8571049481630325, 0.754267284216789)

In [65]:
get_cos_rc(cos_reslut_rc_dc)

(0.9486609116196633, 0.7466261019500403)

In [70]:
#get_cos_rc(cos_reslut_rc_pool)

In [66]:
get_cos_rc(cos_reslut_rc_conv)

(0.9176416501402855, 0.7629800434104907)

## Trans

In [67]:
get_cos_trans(cos_reslut_rc_org)

(0.8066274027411755, 0.7712527845785596)

In [68]:
get_cos_trans(cos_reslut_rc_dc)

(0.7830143533838101, 0.7722905325316469)

In [71]:
#get_cos_trans(cos_reslut_rc_pool)

In [69]:
get_cos_trans(cos_reslut_rc_conv)

(0.8600598149574722, 0.7856783228615919)

## Cos thresholds

In [29]:
thresholds = np.arange(0,1.01,0.01).astype(np.float32)

In [78]:
threshold = 0.7

In [79]:
def get_recall(cos_result,threshold=0.8):
    TP = 0.0
    FN = 0.0
    for cos_res in cos_result:
        trans_ = np.diagonal(cos_res)
        for trans in trans_:
            if trans > threshold:
                TP +=1
            else:
                FN += 1
    return (TP/(TP+FN))

In [80]:
get_recall(cos_reslut_rc_org,threshold)

0.895

In [81]:
get_recall(cos_reslut_rc_dc,threshold),

(0.80625,)

In [83]:
get_recall(cos_reslut_rc_conv,threshold),

(0.96,)